# Nexmart code challenge assignment
The following notebook is divided into 3 sections:
1. Preliminary data exploration
2. Data cleaning and pipeline
3. SQL analysis

Data exploration was included for completeness, but you may skip to part 2 and run the notebook from there to get straight to the assignment. The dashboard itself is located in xx.


## Preliminary Data Analysis (feel free to skip)
The following section serves merely to explore the structure of the data and locate inconsistencies to be cleaned later on.

In [106]:
import pandas as pd

In [107]:
# import dataframes

manufacturers_df = pd.read_csv('../data/manufacturers.csv', delimiter = ";")
prod_description_df = pd.read_csv('../data/product_descriptions.csv', delimiter = ";")
prod_properties_df = pd.read_csv('../data/product_properties.csv', delimiter = ";")

### Manufacturer data

In [108]:
# shows what the name of manufacturer is and what part IDs they manufacture
manufacturers_df.head(5)

,Manufacturernumber,Manufacturername
0,0 601 9K8 000,BOSCH
1,71222062000,FEIN
2,0 601 6B9 000,BOSCH
3,0 601 9K6 100,BOSCH
4,0 601 482 400,BOSCH


In [109]:
# some manufacturers are missing part numbers
manufacturers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Manufacturernumber  285 non-null    object
 1   Manufacturername    309 non-null    object
dtypes: object(2)
memory usage: 5.0+ KB


In [110]:
# only data from 5 manufacturers available
manufacturers_df.groupby('Manufacturername').count()

,Manufacturernumber
Manufacturername,
AAT,0
AIRCRAFT,0
ALMI,0
BEPO,0
BOSCH,117
BRENNENSTUHL,0
BUDAKTOOLS,0
COX,0
FEIN,100


In [111]:
# check if there are duplicates
manufacturers_df.duplicated().sum()

np.int64(0)

### Product descriptions data

In [112]:
# each row shows the short and long descriptions of every product
prod_description_df.head()

,Articlenumber,Short description,Short description 2,Long description,Language
0,S-1124F-A,STYcutter 1124F-A – unser Einstiegsmodell mit ...,NaN,Die STYcutter 1124F-A – unser Einstiegsmodell ...,de
1,S-1310-A,STYcutter 1310-A – Der Allrounder,NaN,Die STYcutter 1310 – Der Allrounder\n\n\nUm d...,de
2,S-1310RW-A,STYcutter 1310RW-A – Der Allrounder,NaN,Die STYcutter 1310RW-A – Der Allrounder\n\n\n...,de
3,S-1310SD-A,STYcutter 1310SD-A – bewährte Qualität,NaN,Um den Anforderungen der Verarbeiter gerecht z...,de
4,S-UL55-22-K,"STYcutter UL55-22-K im Koffer – wenn mobil, da...",NaN,"Die STYcutter UL55-22-K – wenn mobil, dann ric...",de


In [113]:
# most descriptions are present
prod_description_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Articlenumber        470 non-null    object
 1   Short description    466 non-null    object
 2   Short description 2  143 non-null    object
 3   Long description     418 non-null    object
 4   Language             470 non-null    object
dtypes: object(5)
memory usage: 18.5+ KB


In [114]:
# only 2 languages present
prod_description_df.Language.unique()

array(['de', 'en'], dtype=object)

In [115]:
# some articles are repeated in the dataset
prod_description_df.Articlenumber.nunique()

430

In [116]:
# repetition due to language
prod_description_df.groupby('Language')['Articlenumber'].count()

Language
de    430
en     40
Name: Articlenumber, dtype: int64

In [117]:
# check if there are duplicates
prod_description_df.duplicated().sum()

np.int64(0)

In [118]:
# check for NA type values in string columns by filtering strings <5 length
prod_des_less5 = prod_description_df[prod_description_df.applymap(lambda x: len(x)<5 if pd.notna(x) else x)]
prod_des_less5.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Articlenumber        0 non-null      object
 1   Short description    1 non-null      object
 2   Short description 2  0 non-null      object
 3   Long description     0 non-null      object
 4   Language             470 non-null    object
dtypes: object(5)
memory usage: 18.5+ KB


C:\Users\jonah\AppData\Local\Temp\ipykernel_33556\1366483133.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  prod_des_less5 = prod_description_df[prod_description_df.applymap(lambda x: len(x)<5 if pd.notna(x) else x)]


In [119]:
# no NA type valyes for short descriptions, assume informative
prod_des_less5.query('`Short description`.notna()')

,Articlenumber,Short description,Short description 2,Long description,Language
364,NaN,BST,NaN,NaN,de


### Product properties data

In [120]:
# each row should represent a unique product with manufacturernumber and articlenumber as keys
prod_properties_df.head(5)

,Manufacturernumber,Articlenumber,EAN,Technical details,Product category,Technical specifications,Picture normal reduced,Depth m,Width m,Length m,...,Delivery time days,Type of product,Price quantity,ETIM Features,ETIM,ECLASS Features,ECLASS,PROFICLASS Features,PROFICLASS,Product features
0,NaN,S-1124F-A,NaN,§Schnitthöhe§1130§mm|§Schnitttiefe§250§mm|§Geh...,NaN,NaN,'https://www.nexmart.com/media/catalog/ampshar...,0.28,0.67,1.40,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,S-1310-A,NaN,§Schnitthöhe§1130§mm|§Schnitttiefe§310§mm|§Geh...,NaN,NaN,'https://www.nexmart.com/media/catalog/ampshar...,0.28,0.67,1.40,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,S-1310RW-A,NaN,§Schnitthöhe§1130§mm|§Schnitttiefe§310§mm|§Geh...,NaN,NaN,'https://www.nexmart.com/media/catalog/ampshar...,0.28,0.67,1.40,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,S-1310SD-A,NaN,§Schnitthöhe§1130§mm|§Schnitttiefe§310§mm|§Geh...,NaN,NaN,'https://www.nexmart.com/media/catalog/ampshar...,0.28,0.67,1.40,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,S-UL55-22-K,NaN,§Schnitthöhe§550§mm|§Schnitttiefe§220§mm|§Gehr...,NaN,NaN,'https://www.nexmart.com/media/catalog/ampshar...,0.19,0.45,0.86,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
# many variables are either completely missing or have many nulls
# key variables are missing values (manufacturer number and article number)
prod_properties_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Manufacturernumber        285 non-null    object 
 1   Articlenumber             426 non-null    object 
 2   EAN                       342 non-null    float64
 3   Technical details         400 non-null    object 
 4   Product category          0 non-null      float64
 5   Technical specifications  0 non-null      float64
 6   Picture normal reduced    430 non-null    object 
 7   Depth m                   370 non-null    float64
 8   Width m                   372 non-null    float64
 9   Length m                  370 non-null    float64
 10  Weight kg                 425 non-null    float64
 11  Delivery time days        4 non-null      float64
 12  Type of product           218 non-null    object 
 13  Price quantity            430 non-null    int64  
 14  ETIM Featu

In [122]:
# check how many unique values there are for each column
prod_properties_df.apply(lambda x: x.nunique())

Manufacturernumber          285
Articlenumber               426
EAN                         342
Technical details           306
Product category              0
Technical specifications      0
Picture normal reduced      380
Depth m                     115
Width m                     122
Length m                    144
Weight kg                   274
Delivery time days            2
Type of product               1
Price quantity                1
ETIM Features                41
ETIM                          8
ECLASS Features               0
ECLASS                        0
PROFICLASS Features           0
PROFICLASS                    0
Product features              0
dtype: int64

In [123]:
# check what Type of product, ETIM, and ETIM Features looks like
# not all products fall under ETIM classification, perhaps exclude from analysis

pd.concat([prod_properties_df.query("`Type of product`.notna()")["Type of product"].head(5),
           prod_properties_df.query("ETIM.notna()")["ETIM"].head(5),
           prod_properties_df.query("`ETIM Features`.notna()")["ETIM Features"].head(5)
], axis=1)

,Type of product,ETIM,ETIM Features
7,main_product,NaN,NaN
8,main_product,NaN,NaN
9,main_product,NaN,NaN
10,main_product,NaN,NaN
11,main_product,NaN,NaN
282,NaN,EC002689,§Ausführung/Betätigung§akku-hydraulisch§§|§Gee...
283,NaN,EC002689,§Ausführung/Betätigung§akku-hydraulisch§§|§Gee...
284,NaN,EC000168,§Geeignet für§Kabelschuhe und Verbinder§§|§Pre...
286,NaN,EC000168,§Geeignet für§Kabelschuhe und Verbinder§§|§Pre...
287,NaN,EC000168,§Geeignet für§Kabelschuhe und Verbinder§§|§Pre...


In [124]:
# check for NA type values in string columns by filtering strings <5 length
prod_properties_str_df = prod_properties_df.select_dtypes(include=["object", "string"])
prod_prop_less5 = prod_properties_str_df[prod_properties_str_df.applymap(lambda x: len(x)<5 if pd.notna(x) else x)]
prod_prop_less5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Manufacturernumber      0 non-null      object
 1   Articlenumber           0 non-null      object
 2   Technical details       0 non-null      object
 3   Picture normal reduced  4 non-null      object
 4   Type of product         0 non-null      object
 5   ETIM Features           0 non-null      object
 6   ETIM                    0 non-null      object
dtypes: object(7)
memory usage: 23.6+ KB


C:\Users\jonah\AppData\Local\Temp\ipykernel_33556\4130063208.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  prod_prop_less5 = prod_properties_str_df[prod_properties_str_df.applymap(lambda x: len(x)<5 if pd.notna(x) else x)]


In [125]:
# Remove ' during cleaning later

prod_prop_less5.query("`Picture normal reduced`.notna()")

,Manufacturernumber,Articlenumber,Technical details,Picture normal reduced,Type of product,ETIM Features,ETIM
308,NaN,NaN,NaN,',NaN,NaN,NaN
321,NaN,NaN,NaN,',NaN,NaN,NaN
325,NaN,NaN,NaN,',NaN,NaN,NaN
327,NaN,NaN,NaN,',NaN,NaN,NaN


## Assignment task 1: Data cleaning and pipeline
This section does the following:
1. Extract datasets
2. Renaming of variable names and cleaning data (dropping NA keys columns, dropping NA type values like "'")
3. Dropping unused columns and merging into wide dataset
4. Generating mock sales data
5. Preparing datasets for BI dashboard use later
6. Loads data as parquet for BI dashboard use later

In [ ]:
from __future__ import annotations
from pathlib import Path
from typing import Dict, List
import pandas as pd
from pandasql import PandaSQL
import numpy as np

### Extract

In [ ]:
_DATA_PATH = "../data"

def _read_csv(path: Path, delimiter: str = ";") -> pd.DataFrame:
    """Read a CSV file."""
    return pd.read_csv(path, delimiter=delimiter)


def extract_datasets(data_dir: Path | str = _DATA_PATH) -> Dict[str, pd.DataFrame]:
    """Load all raw tables found in *data_dir*."""
    files = {
        "manufacturers": "manufacturers.csv",
        "prod_desc": "product_descriptions.csv",
        "prod_props": "product_properties.csv",
    }
    data_dir = Path(data_dir)
    return {k: _read_csv(data_dir / v) for k, v in files.items()}

### Transform

#### Renaming variables and cleaning

In [128]:
_MANUFACTURER_MAP = {
    "Manufacturernumber": "manufacturer_part_number",
    "Manufacturername": "manufacturer_name",
}

_PROD_DESC_MAP = {
    "Articlenumber": "article_number",
    "Short description": "description_short",
    "Short description 2": "description_short_2",
    "Long description": "description_long",
    "Language": "description_language",
}

_PROD_PROP_MAP = {
    "Manufacturernumber": "manufacturer_part_number",
    "Articlenumber": "article_number",
    "Technical details": "technical_details",
    "Product category": "product_category",
    "Technical specifications": "technical_specifications",
    "Picture normal reduced": "picture_link",
    "Depth m": "dimension_depth_m",
    "Width m": "dimension_width_m",
    "Length m": "dimension_length_m",
    "Weight kg": "dimension_weight_kg",
    "Delivery time days": "delivery_time_days",
    "Type of product": "product_type",
    "Price quantity": "price_quantity",
    "ETIM Features": "etim_features",
    "ETIM": "etim",
    "ECLASS Features": "eclass",
    "PROFICLASS Features": "proficlass_features",
    "PROFICLASS": "proficlass",
    "Product features": "product_features",
}


def _rename_and_drop_null_keys(df: pd.DataFrame, mapping: dict[str, str], keys: List[str]) -> pd.DataFrame:
    """Rename columns then drop rows with null *keys*."""
    df = df.rename(columns=mapping)
    return df.query(" & ".join(f"{k}.notna()" for k in keys))


def clean_manufacturers(df: pd.DataFrame) -> pd.DataFrame:
    """Tidy manufacturers table."""
    return _rename_and_drop_null_keys(df, _MANUFACTURER_MAP, ["manufacturer_part_number", "manufacturer_name"])


def clean_prod_desc(df: pd.DataFrame) -> pd.DataFrame:
    """Tidy product‑descriptions table."""
    return _rename_and_drop_null_keys(df, _PROD_DESC_MAP, ["article_number"])


def clean_prod_props(df: pd.DataFrame) -> pd.DataFrame:
    """Tidy product‑properties table and normalise bad picture links."""
    df = _rename_and_drop_null_keys(df, _PROD_PROP_MAP, ["manufacturer_part_number", "article_number"])
    df.loc[df["picture_link"] == "'", "picture_link"] = np.nan
    return df

#### Drop unnecessary columns and merge

In [129]:
_DROP_COLS = [
    "product_category",
    "technical_specifications",
    "product_type",
    "etim_features",
    "eclass",
    "proficlass",
    "proficlass_features",
    "product_features",
]


def merge_tables(props: pd.DataFrame, desc: pd.DataFrame, man: pd.DataFrame) -> pd.DataFrame:
    """Combine cleaned tables into one wide frame."""
    df = props.merge(desc, on="article_number", how="left")
    df = df.merge(man, on="manufacturer_part_number", how="left")
    return df.drop(columns=_DROP_COLS)

#### Adding mock sales data for dashboard analysis

Assume 10% increase in data quality lead to 10% increase in sales (or more simply perfect proportionality where 1% increase in data quality = 1% increase in sales)

$Y_i = \beta_0*Q_i*\epsilon_i$

where $Y_i$ is the sales revenue (in thousands) from product $i$, $Q$ is the data quality (ranging from 0 to 1) of product $i$, and $\epsilon$ is a random normally distributed error term.

Using this we generate mock sales data for all products.

In [130]:
def generate_mock_revenue(df: pd.DataFrame) -> pd.DataFrame:
    """Add *data_quality* and synthetic *product_revenue*."""
    quality = (
        df.drop(columns=[
            "manufacturer_name",
            "manufacturer_part_number",
            "article_number",
            "description_language",
        ])
        .notna()
        .mean(axis=1)
    )
    df["data_quality"] = quality
    noise = np.random.normal(loc=0.1, scale=0.05, size=len(df))
    df["product_revenue"] = 500 * quality * (1 + noise)
    return df

#### Prepare data for BI dashboard later

In [131]:
def prepare_bi_outputs(df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    """Return tidy frames for BI dashboards."""
    df["product_id"] = (
        df["manufacturer_name"]
        + "_"
        + df["manufacturer_part_number"]
        + "_"
        + df["article_number"]
        + "_"
        + df["description_language"]
    )

    quality_revenue = df[["product_id", "manufacturer_name", "product_revenue", "data_quality"]]

    quality_by_var = (
        df.melt(
            id_vars=[
                "manufacturer_name",
                "manufacturer_part_number",
                "article_number",
                "description_language",
                "product_revenue",
            ],
            var_name="variable_name",
            value_name="variable_value",
        )
        .assign(variable_value=lambda d: d["variable_value"].notna().astype(int))
        [["manufacturer_name", "variable_name", "variable_value"]]
    )

    return {
        "data_quality_and_revenue": quality_revenue,
        "data_quality_by_variable": quality_by_var,
    }

#### Load data into parquet for BI dashboard later

In [132]:
def write_parquet(df: pd.DataFrame, path: Path | str) -> None:
    """Persist *df* as Parquet."""
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(path, index=False)

### Run data pipeline

In [ ]:
def run_pipeline(data_dir: Path | str = _DATA_PATH) -> pd.DataFrame:
    """Execute the full ETL and return the melted SQL frame."""
    raw = extract_datasets(data_dir)
    man = clean_manufacturers(raw["manufacturers"])
    desc = clean_prod_desc(raw["prod_desc"])
    props = clean_prod_props(raw["prod_props"])

    merged = merge_tables(props, desc, man)
    merged = generate_mock_revenue(merged)
    bi_outputs = prepare_bi_outputs(merged)

    out_dir = Path(data_dir)
    for name, frame in bi_outputs.items():
        write_parquet(frame, out_dir / f"{name}.parquet")

    sql_frame = merged.melt(
        id_vars=[
            "manufacturer_name",
            "manufacturer_part_number",
            "article_number",
            "description_language",
            "product_revenue",
        ],
        var_name="variable_name",
        value_name="variable_value",
    )
    return sql_frame

In [134]:
sql_analysis_df = run_pipeline()

In [135]:
# quick look at what the final dataset looks like
sql_analysis_df.head(5)

,manufacturer_name,manufacturer_part_number,article_number,description_language,product_revenue,variable_name,variable_value
0,BOSCH,0 601 6B4 000,06016B4000,de,395.754697,EAN,NaN
1,BOSCH,0 601 6B4 000,06016B4000,en,297.002860,EAN,NaN
2,BOSCH,0 601 9J4 002,06019J4002,de,384.532105,EAN,NaN
3,BOSCH,0 601 9J4 002,06019J4002,en,323.428137,EAN,NaN
4,BOSCH,0 601 9H6 000,06019H6000,de,366.893384,EAN,NaN


## SQL Analysis

The following section answers these questions:
1. Which manufacturers have the biggest improvement potential in their data quality in absolute and relative numbers?
2. What product variable/column (description or property) usually contains data of good quality per manufacturer? And what is the % of good quality records per variable/column and manufacturer?
3. Which other interesting insights did you find? Please give us a short explanation together with the SQL statement used to retrieve it.
4. Bonus qns: If this assignment was part of a real project, what modules/steps/pieces are missing in your deliverable that would be necessary for a production environment?



### 1. Biggest improvement potential by manufacturer

We first look at manufacturers' improvement potential using absolute numbers from the table below. The column `total_variables_missing` refers to the count of the number of data variables that did not have an entry for that manufacturer (note: some variables like `product_type` were dropped from the overall analysis due to their uninformative nature). 

In terms of absolute numbers, Bosch , Fein, and Gustav Klauke have the most missing data points across all their products at 362, 304, and 244 respectively, compared to much smaller numbers for Rothenberger and Fischer.

In [136]:
# Create a PandaSQL instance
pandasql_instance = PandaSQL()

query = """
select
    manufacturer_name,
    sum(case when variable_value is null then 1 else 0 end) as total_variables_missing
from sql_analysis_df
group by manufacturer_name
order by total_variables_missing desc
"""

pandasql_instance(query, locals())

,manufacturer_name,total_variables_missing
0,BOSCH,498
1,FEIN,404
2,GUSTAV KLAUKE GMBH,244
3,ROTHENBERGER,88
4,FISCHER,15


We next look at relative improvement potential across firms from the table below. The variable `relative_variables_missing` refers to the proportion of missing data points across all products for each firm. In this case, Gustav Klauke has the most relative improvement potential as 47% of all its products' data points are missing data, more than twice that of the next firm (Fein) at 23%.

In [137]:
# Define the SQL query
query = """
select
    manufacturer_name,
    round(
        avg(case when variable_value is null then 1 else 0 end), 2
    ) as relative_variables_missing
from sql_analysis_df
group by manufacturer_name
order by relative_variables_missing desc
"""

result_df = pandasql_instance(query, locals())

result_df

,manufacturer_name,relative_variables_missing
0,GUSTAV KLAUKE GMBH,0.38
1,FEIN,0.25
2,ROTHENBERGER,0.24
3,BOSCH,0.22
4,FISCHER,0.19


### 2. Which description or property usually contains data of good quality?

We first look at which single variable has the best quality for each manufacturer from the table below. The variable `variable_completeness` refers to the proportion of data points for a given variable and a given firm that is not missing. In other words, this represents the percentage of data points that are 'good'. The variables shown for each firm are the ones each firm does best in, with a 100% completion rate. Firms generally do well in filling in information on price quantity and technical details.

Note: Expanding the table to include the top_n shows that all firms have a few similar variables at 100%.

In [138]:
# Define the SQL query
query = """
with
    aggregate_completeness as (
        select
            manufacturer_name,
            variable_name,
            round(
                avg(case when variable_value is null then 0 else 1 end), 2
            ) as variable_completeness
        from sql_analysis_df
        group by manufacturer_name, variable_name
        order by manufacturer_name, variable_completeness desc
    ),

    ranked as (
        select
            *,
            row_number() over (
                partition by manufacturer_name order by variable_completeness desc
            ) as rn
        from aggregate_completeness
    )

select manufacturer_name, variable_name, variable_completeness
from ranked
where rn = 1

"""

pandasql_instance(query, locals())

,manufacturer_name,variable_name,variable_completeness
0,BOSCH,product_id,1.0
1,FEIN,technical_details,1.0
2,FISCHER,technical_details,1.0
3,GUSTAV KLAUKE GMBH,technical_details,1.0
4,ROTHENBERGER,technical_details,1.0


Note: Pivoting is cumbersome in SQL, and so I am showing only a sample of variables here.

We next look at what data quality looks like for a few select variables for all manufacturers from the table below. We see large variation in how firms some variables. For instance, variables like `price_quantity` or `picture_link` are almost always filled in for all firms, but either highly present or not at all for `description_short_2`. Delivery time information is also missing for most products across all firms.

In [139]:
# Define the SQL query

# manually pivoting the table using sql since the table is in long format
query = """
with
    aggregate_completeness as (
        select
            manufacturer_name,
            variable_name,
            round(
                avg(case when variable_value is null then 0 else 1 end), 2
            ) as variable_completeness
        from sql_analysis_df
        group by manufacturer_name, variable_name
    )

select
    manufacturer_name,
    sum(
        case
            when variable_name = 'delivery_time_days' then variable_completeness else 0
        end
    ) as delivery_time_days,
    sum(
        case
            when variable_name = 'description_long' then variable_completeness else 0
        end
    ) as description_long,
    sum(
        case
            when variable_name = 'description_short_2' then variable_completeness else 0
        end
    ) as description_short_2,
    sum(
        case
            when variable_name = 'dimension_depth_m' then variable_completeness else 0
        end
    ) as dimension_depth_m,
    sum(
        case
            when variable_name = 'dimension_weight_kg' then variable_completeness else 0
        end
    ) as dimension_weight_kg,
    sum(
        case when variable_name = 'picture_link' then variable_completeness else 0 end
    ) as picture_link,
    sum(
        case when variable_name = 'price_quantity' then variable_completeness else 0 end
    ) as price_quantity
from aggregate_completeness
group by manufacturer_name
"""

pandasql_instance(query, locals())

,manufacturer_name,delivery_time_days,description_long,description_short_2,dimension_depth_m,dimension_weight_kg,picture_link,price_quantity
0,BOSCH,0.00,0.8,0.94,0.97,1.00,1.0,1.0
1,FEIN,0.00,1.0,0.00,1.00,0.96,1.0,1.0
2,FISCHER,0.00,1.0,1.00,1.00,1.00,1.0,1.0
3,GUSTAV KLAUKE GMBH,0.00,1.0,0.00,0.00,1.00,0.9,1.0
4,ROTHENBERGER,0.17,1.0,0.00,1.00,1.00,1.0,1.0


### 3. Interesting insights

I was interested in two things based on the data:
1. Rather than at the manufacturer or variable level, what does data quality look like at the product level and how much does that vary within firms? 
2. Are more expensive products are better labelled? In the absence of pricing information, I use weight to proxy for price, with the assumption that heavier products should also be costlier.

The SQL query below shows the average variable completeness for each firm across its products (which is simply the inverse of the output in question 1 part 2) and also the standard deviation in data quality across products offered by each firm. To illustrate, on average Fischer's products have 'good' quality data for 85% of the available fields, and the variation across products for this is negligible. On the other hand, Bosch's products only have 80% for this quantity and it deviates by 7% on average from product to product.

In [140]:
# Define the SQL query
query = """
with
    aggregate_completeness as (
        select
            manufacturer_name,
            manufacturer_part_number,
            article_number,
            description_language,
            avg(
                case when variable_value is null then 0 else 1 end
            ) as variable_completeness
        from sql_analysis_df
        group by
            manufacturer_name,
            manufacturer_part_number,
            article_number,
            description_language
    )

select
    manufacturer_name,
    round(avg(variable_completeness), 2) as mean_variable_completeness,
    round(
        sqrt(
            (
                sum(variable_completeness * variable_completeness)
                - sum(variable_completeness) * sum(variable_completeness) / count(*)
            )
            / count(*)
        ),
        2
    ) as stdev_variable_completeness
from aggregate_completeness
group by manufacturer_name
order by mean_variable_completeness desc
"""

pandasql_instance(query, locals())

,manufacturer_name,mean_variable_completeness,stdev_variable_completeness
0,FISCHER,0.81,0.00
1,BOSCH,0.78,0.05
2,ROTHENBERGER,0.76,0.02
3,FEIN,0.75,0.01
4,GUSTAV KLAUKE GMBH,0.62,0.02


For the query below, I order products in terms of quantiles, with the expectation that products in the highest quantile (and assumedly also the most expensive) should have higher data quality. The converse seems to be true, where lighter products appear to have better data quality.

In [141]:
query = """
with
    get_weight as (
        select distinct
            manufacturer_name,
            manufacturer_part_number,
            article_number,
            cast(variable_value as float) as dimension_weight_kg
        from sql_analysis_df
        where variable_name = 'dimension_weight_kg'
    ),

    get_weight_order as (
        select *, row_number() over (order by dimension_weight_kg) as rn
        from get_weight
        where dimension_weight_kg is not null
    ),

    get_total_rows as (
        select
            *,
            (
                select count(*) from get_weight where dimension_weight_kg is not null
            ) as total_rows
        from get_weight_order
    ),

    get_weight_quantiles as (
        select
            *,
            case
                when rn <= total_rows / 5
                then 'Q1'
                when rn <= 2 * total_rows / 5
                then 'Q2'
                when rn <= 3 * total_rows / 5
                then 'Q3'
                when rn <= 4 * total_rows / 5
                then 'Q4'
                else 'Q5'
            end as weight_quantile
        from get_total_rows
    ),

    join_data as (
        select *
        from sql_analysis_df
        join
            get_weight_quantiles using (
                manufacturer_name, manufacturer_part_number, article_number
            )
    )

select
    weight_quantile,
    round(
        avg(case when variable_value is null then 0 else 1 end), 2
    ) as variable_completeness
from join_data
group by weight_quantile
"""

pandasql_instance(query, locals())

,weight_quantile,variable_completeness
0,Q1,0.78
1,Q2,0.77
2,Q3,0.73
3,Q4,0.74
4,Q5,0.72


I compute the Pearson's R correlation between the two variables below and find that there is indeed a slight negative correlation.

In [142]:
query = """
with
    get_weight as (
        select distinct
            manufacturer_name,
            manufacturer_part_number,
            article_number,
            cast(variable_value as float) as dimension_weight_kg
        from sql_analysis_df
        where variable_name = 'dimension_weight_kg'
    ),

    get_weight_order as (
        select *, row_number() over (order by dimension_weight_kg) as rn
        from get_weight
        where dimension_weight_kg is not null
    ),

    get_total_rows as (
        select
            *,
            (
                select count(*) from get_weight where dimension_weight_kg is not null
            ) as total_rows
        from get_weight_order
    ),

    get_weight_quantiles as (
        select
            *,
            case
                when rn <= total_rows / 5
                then 'Q1'
                when rn <= 2 * total_rows / 5
                then 'Q2'
                when rn <= 3 * total_rows / 5
                then 'Q3'
                when rn <= 4 * total_rows / 5
                then 'Q4'
                else 'Q5'
            end as weight_quantile
        from get_total_rows
    ),

    join_data as (
        select *
        from sql_analysis_df
        join
            get_weight_quantiles using (
                manufacturer_name, manufacturer_part_number, article_number
            )
    ),

    weight_and_completeness as (
        select
            manufacturer_name,
            manufacturer_part_number,
            article_number,
            dimension_weight_kg,
            round(
                avg(case when variable_value is null then 0 else 1 end), 2
            ) as variable_completeness
        from join_data
        group by
            manufacturer_name,
            manufacturer_part_number,
            article_number,
            dimension_weight_kg
    )

select
    (
        count(*) * sum(dimension_weight_kg * variable_completeness)
        - sum(dimension_weight_kg) * sum(variable_completeness)
    ) / (
        sqrt(
            count(*) * sum(dimension_weight_kg * dimension_weight_kg)
            - sum(dimension_weight_kg) * sum(dimension_weight_kg)
        ) * sqrt(
            count(*) * sum(variable_completeness * variable_completeness)
            - sum(variable_completeness) * sum(variable_completeness)
        )
    ) as pearsons_r
from weight_and_completeness
where dimension_weight_kg is not null and variable_completeness is not null
"""

pandasql_instance(query, locals())

,pearsons_r
0,-0.285493


### Bonus question: What is missing for a production environment?

1. Orchestration: the pipeline should run periodically on new incoming data using an orchestrator tool (dbt, Airflow, etc).
2. Data quality and freshness tests: automatic checks that test if input data is in correct format, variables of acceptable data types, checks for null values, and if data is is up to date (fresh).
3. Data storage: raw data stored in a database (rather than local parquet files) like BigQuery or Snowflake
4. Data catalog: a centralized catalog that documents variables in each dataset, relationship to each other, and also contains informative descriptions of what each variable represents.
5. Refactor code to follow naming convention of codebase